In [10]:
import time
from datetime import datetime
import hmac
import hashlib
from typing import Dict, Tuple
import os
from dotenv import load_dotenv

load_dotenv()


class QueueAndTradeManager:
    def __init__(self, api_key: str, api_secret: str) -> None:
        self.enable_trade = False
        if api_key is None or api_secret is None:
            raise ValueError("api_key or api_secret is None. Check your .env file.")

        self.api_key = api_key
        self.api_secret = api_secret
        self.http_request_private_baseurl = "https://api.coin.z.com/private"

    def __del__(self):
        import time

        # Sometime, Broken pipe error raises becase main process finishes faster than Queue.close().
        time.sleep(0.01)

    def http_headers(self, method: str, endpint: str):
        timestamp = "{0}000".format(int(time.mktime(datetime.now().timetuple())))

        text = timestamp + method + endpint
        sign = hmac.new(bytes(self.api_secret.encode("ascii")), bytes(text.encode("ascii")), hashlib.sha256).hexdigest()

        headers = {"API-KEY": self.api_key, "API-TIMESTAMP": timestamp, "API-SIGN": sign}
        return headers

    def test_http_private_request_args(self) -> Tuple[str, Dict]:
        """Get args to HTTP private request for GMO exchange

        Returns:
            Tuple[str, Dict]: uri and headers
        """
        endpoint = "/v1/account/margin"
        return self.http_request_private_baseurl + endpoint, self.http_headers(method="GET", endpint=endpoint)

In [11]:
manager = QueueAndTradeManager(api_key=os.environ["EXCHANGE_API_KEY"], api_secret=os.environ["EXCHANGE_API_SECRET"])

url, headers = manager.test_http_private_request_args()
url, headers

('https://api.coin.z.com/private/v1/account/margin',
 {'API-KEY': 'Plfxvm+FGFajlkeujJLhlzryVE5TkEJl',
  'API-TIMESTAMP': '1646646488000',
  'API-SIGN': '53fdef3a688422ef358b5a028e9f815504120422ced3038977ca6a8843ab27b2'})

In [12]:
import requests

requests.get(url, headers=headers)

<Response [200]>

In [9]:
import requests
import json
import hmac
import hashlib
import time
from datetime import datetime

apiKey    = os.environ["EXCHANGE_API_KEY"]
secretKey = os.environ["EXCHANGE_API_SECRET"]

timestamp = '{0}000'.format(int(time.mktime(datetime.now().timetuple())))
method    = 'GET'
endPoint  = 'https://api.coin.z.com/private'
path      = '/v1/account/margin'

text = timestamp + method + path
sign = hmac.new(bytes(secretKey.encode('ascii')), bytes(text.encode('ascii')), hashlib.sha256).hexdigest()

headers = {
    "API-KEY": apiKey,
    "API-TIMESTAMP": timestamp,
    "API-SIGN": sign
}

requests.get(endPoint+path, headers=headers)

<Response [200]>